In [1]:
# Standard library imports
import os
import json
import gc
import logging
import time
from multiprocessing import Pool, cpu_count, current_process, Manager, get_context

# Third-party library imports
import torch
from PIL import Image
from torchvision.transforms import ToPILImage
from transformers import pipeline
from diffusers import DiffusionPipeline
from diffusers.models.modeling_outputs import Transformer2DModelOutput
from IPython.display import display
import warnings

# Local/application-specific imports
import bittensor as bt
from bitmind.constants import PROMPT_GENERATOR_NAMES, PROMPT_GENERATOR_ARGS, DIFFUSER_NAMES, DIFFUSER_ARGS
from multiprocessing_tasks import worker_initializer, generate_images_for_chunk

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow logging (1: filter out INFO, 2: additionally filter out WARNING, 3: additionally filter out ERROR)
import tensorflow as tf  # Import TensorFlow after setting the log level

2024-07-04 17:17:46.216329: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-04 17:17:46.241785: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-04 17:17:46.720352: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/saccharinedreams/miniconda3/envs/bitmind/lib/python3.10/site-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` fr

2024-07-04 17:17:48.477 |       INFO       |  - Loading image generation model (stabilityai/stable-diffusion-xl-base-1.0)... - 
2024-07-04 17:19:05.851 |       INFO       |  - Loading image generation model (SG161222/RealVisXL_V4.0)... - 


In [2]:
# Configure logging
logging.basicConfig(level=logging.INFO)
# Suppress FutureWarnings from diffusers module
warnings.filterwarnings("ignore", category=FutureWarning, module='diffusers')
# Set device for model operations
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cpu":
    raise RuntimeError("This script requires a GPU because it uses torch.float16.")  # Added check for GPU availability
# Ensure that this script uses 'spawn' method for starting multiprocessing tasks
ctx = get_context("spawn")

In [3]:
def list_datasets(base_dir):
    """List all subdirectories in the base directory."""
    return [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

def load_annotations(base_dir, dataset):
    """Load annotations from JSON files within a specified directory."""
    annotations = []
    path = os.path.join(base_dir, dataset)
    for filename in os.listdir(path):
        if filename.endswith(".json"):
            with open(os.path.join(path, filename), 'r') as file:
                data = json.load(file)
                annotations.append(data)
    return annotations

def load_diffuser(model_name):
    """Load a diffusion model by name, configured according to provided arguments."""
    bt.logging.info(f"Loading image generation model ({model_name})...")
    model = DiffusionPipeline.from_pretrained(
        model_name, torch_dtype=torch.float32 if device == "cpu" else torch.float16, **DIFFUSER_ARGS[model_name]
    )
    model.to(device)
    return model

In [4]:
## GPU
def generate_images(annotations, diffuser, save_dir, num_images, batch_size):
    """Generate images from annotations using a diffuser and save to directory."""
    os.makedirs(save_dir, exist_ok=True)
    generated_images = []
    start_time = time.time()

    # Process in batches
    num_batches = (len(annotations) + batch_size - 1) // batch_size

    with torch.no_grad():
        for i in range(min(num_images, len(annotations))):
            start_loop = time.time()
            annotation = annotations[i]
            prompt = annotation['description']
            logging.info(f"Annotation {i}: {json.dumps(annotation, indent=2)}")
            
            # Generate image tensor
            generated_image = diffuser(prompt=prompt).images[0]
            logging.info(f"Type of generated image: {type(generated_image)}")

            # Check if conversion to PIL image is necessary
            if isinstance(generated_image, torch.Tensor):
                img = ToPILImage()(generated_image)
            else:
                img = generated_image  # No conversion needed

            img_filename = f"{save_dir}/{prompt[:50].replace(' ', '_')}-{i}.png"
            img.save(img_filename)
            generated_images.append(img_filename)
            loop_time = time.time() - start_loop
            logging.info(f"Image saved to {img_filename}")

    total_time = time.time() - start_time
    logging.info(f"Total processing time: {total_time:.2f} seconds")
    return generated_images


def test_diffusers_on_datasets(annotations_dir, output_dir, num_images=1, batch_size=2):
    """Test various diffusers on datasets."""
    datasets = list_datasets(annotations_dir)
    for dataset in datasets:
        annotations = load_annotations(annotations_dir, dataset)
        diffuser = None
        for diffuser_name in DIFFUSER_NAMES:
            if diffuser is not None:
                logging.info("Deleting previous diffuser, freeing memory")
                diffuser.to('cpu')
                del diffuser
                gc.collect()
                torch.cuda.empty_cache()

            logging.info(f"Testing {diffuser_name} on annotation dataset {dataset}...")
            diffuser = load_diffuser(diffuser_name)
            try:
                save_dir = os.path.join(output_dir, dataset)
                generated_images = generate_images(
                    annotations, diffuser, save_dir, num_images=num_images, batch_size=batch_size
                )
                logging.info("Images generated and saved successfully.\n")
            except Exception as e:
                logging.error(f"Failed to generate image with {diffuser_name}: {str(e)}\n")

In [6]:
## Multiprocessing loop
def multiprocess_generate_images(annotations_dir, output_dir, num_processes=None):
    if num_processes is None:
        num_processes = max(1, cpu_count() - 1)  # Leaves one CPU core free

    datasets = list_datasets(annotations_dir)
    for model_name in DIFFUSER_NAMES:
        logging.info(f"Processing with model: {model_name}")
        with ctx.Pool(processes=num_processes, initializer=worker_initializer, initargs=(model_name, device, DIFFUSER_ARGS)) as pool:
            for dataset in datasets:
                annotations = load_annotations(annotations_dir, dataset)
                save_dir = os.path.join(output_dir, model_name, dataset)

                # Split annotations into chunks for each worker
                chunk_size = (len(annotations) + num_processes - 1) // num_processes
                chunks = [annotations[i:i + chunk_size] for i in range(0, len(annotations), chunk_size)]

                results = pool.starmap(generate_images_for_chunk, [(chunk, save_dir) for chunk in chunks])
                logging.info(f"Completed processing for dataset {dataset} with model {model_name}")

In [7]:
ANNOTATIONS_DIR = "annotations/"
OUTPUT_DIR = "synthetics_from_annotations/"

In [8]:
# GPU
test_diffusers_on_datasets(ANNOTATIONS_DIR, OUTPUT_DIR, num_images=1, batch_size=2)

INFO:root:Testing stabilityai/stable-diffusion-xl-base-1.0 on annotation dataset dalle-mini_open-images...
INFO:bittensor: - Loading image generation model (stabilityai/stable-diffusion-xl-base-1.0)... - 


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

INFO:root:Annotation 0: {
  "description": "A picture of a group of people playing in a yard.The setting is a grassy area with a red ball and a few people.The background is a green lawn.The people are wearing blue shirts and black pants.."
}


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:root:Type of generated image: <class 'PIL.Image.Image'>
INFO:root:Image saved to synthetics_from_annotations/dalle-mini_open-images/A_picture_of_a_group_of_people_playing_in_a_yard.T-0.png
INFO:root:Total processing time: 69.10 seconds
INFO:root:Images generated and saved successfully.

INFO:root:Deleting previous diffuser, freeing memory
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argument, or use another device for inference.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operat

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

INFO:root:Annotation 0: {
  "description": "A picture of a group of people playing in a yard.The setting is a grassy area with a red ball and a few people.The background is a green lawn.The people are wearing blue shirts and black pants.."
}


  0%|          | 0/50 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # GPU                                                                                        │
│ ❱ 2 test_diffusers_on_datasets(ANNOTATIONS_DIR, OUTPUT_DIR, num_images=1, batch_size=2)          │
│   3                                                                                              │
│                                                                                                  │
│ in test_diffusers_on_datasets:57                                                                 │
│                                                                                                  │
│   54 │   │   │   diffuser = load_diffuser(diffuser_name)                                         │
│   55 │   │   │   try:                                                                            │
│   56 │   │   │   │   save_dir = os.path.join(output_dir, dataset)                                │
│ ❱ 57 │   │   │   │   generated_images = generate_images(                                         │
│   58 │   │   │   │   │   annotations, diffuser, save_dir, num_images=num_images, batch_size=b    │
│   59 │   │   │   │   )                                                                           │
│   60 │   │   │   │   logging.info("Images generated and saved successfully.\n")                  │
│                                                                                                  │
│ in generate_images:19                                                                            │
│                                                                                                  │
│   16 │   │   │   logging.info(f"Annotation {i}: {json.dumps(annotation, indent=2)}")             │
│   17 │   │   │                                                                                   │
│   18 │   │   │   # Generate image tensor                                                         │
│ ❱ 19 │   │   │   generated_image = diffuser(prompt=prompt).images[0]                             │
│   20 │   │   │   logging.info(f"Type of generated image: {type(generated_image)}")               │
│   21 │   │   │                                                                                   │
│   22 │   │   │   # Check if conversion to PIL image is necessary                                 │
│                                                                                                  │
│ /home/saccharinedreams/miniconda3/envs/bitmind/lib/python3.10/site-packages/torch/utils/_context │
│ lib.py:115 in decorate_context                                                                   │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /home/saccharinedreams/miniconda3/envs/bitmind/lib/python3.10/site-packages/diffusers/pipelines/ │
│ stable_diffusion_xl/pipeline_stable_diffusion_xl.py:1209 in

In [ ]:
# GPU
test_diffusers_on_datasets(ANNOTATIONS_DIR, OUTPUT_DIR, num_images=1, batch_size=4)

In [ ]:
# GPU
test_diffusers_on_datasets(ANNOTATIONS_DIR, OUTPUT_DIR, num_images=1, batch_size=8)

In [ ]:
# GPU
test_diffusers_on_datasets(ANNOTATIONS_DIR, OUTPUT_DIR, num_images=1, batch_size=16)

In [ ]:
# CPU Multiprocessing
# multiprocess_generate_images(ANNOTATIONS_DIR, OUTPUT_DIR)

In [ ]:
#### To-do

-Improve latency for image generation, implementing multiprocessing, or ensure efficient gpu usage

-Set up evaluation for real image counterpart and synthetic generated from annotation of said real image